In [1]:
pip install openpyxl

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 2.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from itertools import groupby
import zipfile
import datetime
import os
from copy import deepcopy
from multiprocessing import Pool
import time
from random import shuffle, seed, sample

seed(3407)


def best_divisor(dividend, divisor_range, mod_range):
    div, mod = divmod(dividend, divisor_range)
    is_available = np.where((mod == 0) | ((mod >= mod_range[0]) & (mod <= mod_range[1])), 1, 0)
    div_ceil = div + np.where(mod > 0, 1, 0)
    out = sorted(zip(is_available, div_ceil, mod, divisor_range), key=lambda x: [-x[0], -x[3]])
    return out[0][-1]


class ALNS:
    def __init__(self, df):
        self.attribute = ["计划日期", "车型", "天窗", "外色描述", "四驱车", "K3", "小颜色", "大颜色", "双色车", "石墨电池", "车辆等级描述", "电池特征", "order"]
        self.gap_limit = {"小颜色": 60, "双色车": 60, "石墨电池": 30}
        self.batch_limit = {"小颜色": [15, 30], "双色车": [0, 4], "大颜色": [15, 9999], "石墨电池": [0, 1]}
        self.df = deepcopy(df)
        self.group, self.data = {}, {}
        for var in self.attribute:
            self.group[var] = np.array(df[var])
            self.data[var] = np.array(df[[var, "num"]])
        self.objective_artificial = self.objective(np.argsort(df["每日顺序"]).tolist())
        df = df.groupby(self.attribute, as_index=False)["num"].sum()
        self.group, self.data = {}, {}
        for var in self.attribute:
            self.group[var] = np.array(df[var])
            self.data[var] = np.array(df[[var, "num"]])
        path = list(range(len(df)))
        self.paths = [path, path[::-1]] + [sample(path, len(path)) for i in range(99)]
        self.score = [self.statistic(path) for path in self.paths]

    def switch_num(self, path, name):
        return len(list(groupby(self.group[name][path]))) - 1

    def batch_objective(self, path, name):
        batch, batch_num = [], []
        for k, g in groupby(self.data[name][path], key=lambda x: x[0]):
            batch.append(k)
            batch_num.append(sum([v[1] for v in g]))
        batch_num, batch = np.array(batch_num), np.array(batch)
        if batch[0] == "other":
            batch_num[0] = 999
        if batch[-1] == "other":
            batch_num[-1] = 999
        n_batch = len(batch)
        gap, num = np.ones(n_batch), np.zeros(n_batch)
        batch_low, batch_up = self.batch_limit[name]
        num = np.where(batch == "other", 1, num)
        num = np.where((batch_num >= batch_low) & (batch_num <= batch_up), 1, num)
        if name in self.gap_limit:
            gap_low = self.gap_limit[name]
            num_left = np.concatenate(([999], batch_num[:-1]))
            num_right = np.concatenate((batch_num[1:], [999]))
            batch_left = np.concatenate((["other"], batch[:-1]))
            batch_right = np.concatenate((batch[1:], ["other"]))
            gap = np.where((batch_left != "other") | (batch_right != "other") | (num_left < gap_low) | (num_right < gap_low), 0, gap)
            gap = np.where(batch == "other", 1, gap)
        idx = np.where(batch != "other")[0]
        if len(idx):
            return np.mean(gap[idx]), np.mean(num[idx])
        else:
            return 1, 1

    def batch_concentrate(self, path, name):
        return len(list([k for k, g in groupby(self.group[name][path]) if k != 'other']))

    def objective(self, path):
        switch = {key: self.switch_num(path, key) for key in ['车型', '天窗', '外色描述', '车辆等级描述', '电池特征']}
        concentrate = {key: self.batch_concentrate(path, key) for key in ['四驱车', 'K3']}
        gap, num = {}, {}
        for key in ['小颜色', '双色车', '大颜色', '石墨电池']:
            gap[key], num[key] = self.batch_objective(path, key)
        objective = {"switch": switch, "concentrate": concentrate, "gap": gap, "num": num}
        return objective

    def statistic(self, path):
        objective = self.objective(path)
        for k1 in objective:
            for k2 in objective[k1]:
                base = self.objective_artificial[k1][k2]
                if k1 in ['switch', 'concentrate']:
                    objective[k1][k2] = (base - objective[k1][k2]) / base if base > 0 else base - objective[k1][k2]
                else:
                    objective[k1][k2] = (objective[k1][k2] - base) / base if base > 0 else np.exp(np.exp(objective[k1][k2] - base))
        return [objective["switch"]["车型"],
                4 * objective["switch"]["天窗"] + 2 * objective["switch"]["外色描述"] +
                objective["concentrate"]["四驱车"] + objective["concentrate"]["K3"] +
                sum(objective["gap"].values()) + sum(objective["num"].values()),
                objective["switch"]["车辆等级描述"] + objective["switch"]["电池特征"]]

    def intra_optimize(self):
        for idx, path in enumerate(self.paths):
            paths = []
            path = [0] + path + [0]
            n = len(path)
            for i in range(0, n - 1):
                for j in range(i + 1, n - 1):
                    paths.append(path[:i + 1] + path[j:i:-1] + path[j + 1:])
            paths = list(set(tuple(path) for path in paths))
            paths = [list(path[1:-1]) for path in paths]
            shuffle(paths)
            score = [self.statistic(path) for path in paths]
            best_id = score.index(sorted(score)[-1])
            self.score[idx], self.paths[idx] = score[best_id], paths[best_id]

    def result(self):
        df_summary = pd.DataFrame(self.group).iloc[self.paths[self.score.index(sorted(self.score)[-1])]].reset_index(drop=True)
        df_summary["rank"] = range(len(df_summary))
        df = self.df.merge(df_summary, how="left", left_on=self.attribute, right_on=self.attribute)
        df = df.sort_values(by=["rank", "每日顺序", "车辆等级描述", "电池特征"]).reset_index(drop=True)
        df = df.drop(["rank", "四驱车", "K3", "小颜色", "大颜色", "双色车", "石墨电池", "order", "num"], axis=1)
        return df


class APS:
    def __init__(self):
        self.small_color = ["量子红", "量子红-Y", "冰玫粉", "冰玫粉-Y", "蒂芙尼蓝", "星漫绿", "星漫绿-Y", "琉璃红", "夜荧黄", "黄绿荧", "薄荷贝绿", "烟雨青", "幻光紫", "广交红",
                            "闪电橙", "脉冲蓝", "天际灰", "火焰橙", "幻光紫", "幻光紫-Y", "琉璃红", "松花黄", "松花黄-Y"]
        self.big_color = ["白云蓝", "极地白", "极地白-Y", "幻影银", "幻影银(出租车)", "极速银", "极速银-Y", "极速银(出租车)", "夜影黑", "夜影黑-Y", "自由灰", "自由灰-Y", "素雅灰",
                          "素雅灰-Y", "天青色", "天青色-Y", "珍珠白", "全息银"]
        self.attribute = ["计划日期", "车型", "天窗", "外色描述", "四驱车", "K3", "小颜色", "大颜色", "双色车", "石墨电池", "车辆等级描述", "电池特征"]
        self.div_dict = {"小颜色": [30] * 2000, "大颜色": [30] * 2000, "双色车": [4] * 2000}
        for i in range(0, 2000):
            self.div_dict['小颜色'][i] = best_divisor(i, np.arange(30, 14, -1), [15, 30])
            self.div_dict['大颜色'][i] = best_divisor(i, np.arange(30, 14, -1), [15, np.inf])
            self.div_dict['双色车'][i] = best_divisor(i, np.arange(4, 0, -1), [0, 4])

    def prepare_data(self):
        df = pd.read_excel('./data/data211909/数据集A榜基于多目标、多源数据扰动预测的智能排序算法.xlsx', sheet_name='处理后数据', header=1, engine='openpyxl')
        df["四驱车"] = np.where(df["电池特征"].str.contains("/"), df["电池特征"], "other")
        df["K3"] = np.where(df["车型"].isin(["K3"]), df["车型"], "other")
        df["小颜色"] = np.where(df["外色描述"].isin(self.small_color), df["外色描述"], "other")
        df["大颜色"] = np.where(df["外色描述"].isin(self.big_color), df["外色描述"], "other")
        df["双色车"] = np.where(df["外色描述"].str.contains("/"), df["外色描述"], "other")
        df["石墨电池"] = np.where(df["电池特征"].str.contains("石墨"), df["电池特征"], "other")
        df = df.sort_values(by=["计划日期", "车型", "天窗", "四驱车", "K3", "外色描述", '车辆等级描述', '电池特征', "每日顺序"]).reset_index(drop=True)
        df["num"] = df.groupby(self.attribute)["每日顺序"].transform("count").values
        df["rank"] = df.groupby(self.attribute)["每日顺序"].transform("rank").values
        df["order"] = np.ceil(df["rank"] / 30)
        df["order"] = np.where(df["外色描述"].isin(self.big_color),
                               np.ceil(df["rank"] / df["num"].apply(lambda x: self.div_dict["大颜色"][int(x)])),
                               df["order"])
        df["order"] = np.where(df["外色描述"].isin(self.small_color),
                               np.ceil(df["rank"] / df["num"].apply(lambda x: self.div_dict["小颜色"][int(x)])),
                               df["order"])
        df["order"] = np.where(df["外色描述"].str.contains("/"),
                               np.ceil(df["rank"] / df["num"].apply(lambda x: self.div_dict["双色车"][int(x)])),
                               df["order"])
        df["order"] = np.where(df["电池特征"].str.contains("石墨"), df["rank"], df["order"])
        df["num"] = 1
        df = df.drop(["rank"], axis=1)
        return df

    def opt(self, g_data):
        alns = ALNS(g_data)
        start_time = time.time()
        early_stop = 0
        while True:
            score_before = sorted(alns.score)[-1]
            alns.intra_optimize()
            score_after = sorted(alns.score)[-1]
            end_time = time.time()
            if end_time - start_time > 360:
                break
            if score_before == score_after:
                early_stop += 1
                if early_stop == 5:
                    break
            else:
                early_stop = 0
        print(f"{g_data['计划日期'].unique()} batch num:{len(alns.paths[-1])},"
              f" elapsed:{np.round(end_time - start_time, 3)}s, "
              f"objective:{np.round(score_after, 3)}"
              )
        return alns.result()

    def run(self):
        df = self.prepare_data()
        param = []
        for date, g_data in df.groupby("计划日期"):
            param.append(g_data)
        p = Pool(min(len(param), os.cpu_count()))
        with p:
            result = p.map(self.opt, param)
        p.close()
        p.join()
        result = pd.concat(result, sort=False, ignore_index=True)
        now = datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")
        path = f"out/{now}"
        os.makedirs(path)
        os.chdir(path)
        result.to_csv("commit.csv", index=False)
        zip_file = zipfile.ZipFile('commit.zip', 'w')
        zip_file.write("commit.csv", compress_type=zipfile.ZIP_DEFLATED)
        zip_file.close()


if __name__ == '__main__':
    aps = APS()
    aps.run()
